In [ ]:
import numpy as np
import cv2, os, argparse, glob
from cone_code import ConeFinder, ConeSeeker

In [ ]:
class Args(object):
    # Set to True to print debug messages
    debug = False
    # Set image_dir to some value to test only images
    # Set image_dir to '' to test with videos
    image_dir = ''
    # Use the video file number or the whole file name (/dev/videoN or captured video file)
    # For realsense, add +2 since it seems to be the third among the video devices that get created
    video_file = ''
    # Set to output file to save the marked video to
    out_file = ''
    codec = '-1'

args = Args()
cf = ConeFinder()
cs = ConeSeeker()

In [ ]:
def find_in_video(fileName):
    cap = cv2.VideoCapture(fileName)
    if(cap.isOpened() == False):
        print("Error: Could not open video file %s. Using default device." % fileName)
        cap = cv2.VideoCapture(0)

    if(cap.isOpened() == False):
      print("Could not open default video device")
      return

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Display the resulting frame
        if(ret):
            #print('---------------------------------')
            poses, listOfCones = cf.find_cones(frame)
            imghull = frame.copy()
            count = len(poses)
            if(count):
                # Frame big 3 cones - they are sorted by area
                cv2.drawContours(imghull, listOfCones[0:2], -1, (0, 255, 0), 3)
                (cl, conf, sd, td) = cs.seek_cone(poses)
                #frame should be 0
                (ih, iw) = imghull.shape[:2]
                pt1 = (int(iw/2 + cl.x - cl.w/2), ih - cl.y)
                pt2 = (int(iw/2 + cl.x + cl.w/2), ih - cl.y - cl.h)
                cv2.rectangle(imghull, pt1, pt2, (0,0,255), 3)
                msg_str = '%.3f' % conf
                cv2.putText(imghull, msg_str, pt1, cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (255,0,0), 2, cv2.LINE_AA)
        
            #cv2.imshow('input', frame)
            cv2.imshow('output', imghull)
            #print('Found %d Cones' % count)

        k = cv2.waitKey(5) & 0xff
        if k == 27:
            break
        if k == ord(' '):
            k = ord('.')
            while k is not ord(' '):
                k = cv2.waitKey(10) & 0xff

    # When everything done, release the capture
    cap.release()        
    cv2.destroyAllWindows()

In [ ]:
args.debug = True

# For Intel realsense, add +2 to video_file location for Linux
# Running on Linux, use /dev/videoN instead where N is the number of device
find_in_video('../videos/forDemo.avi')

print("Done")